# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

[2025-05-08 02:28:39] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=32325, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1071020947, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download

[2025-05-08 02:28:48] Attention backend not set. Use fa3 backend by default.
[2025-05-08 02:28:48] Init torch distributed begin.


[2025-05-08 02:28:49] Init torch distributed ends. mem usage=0.00 GB
[2025-05-08 02:28:49] Load weight begin. avail mem=53.74 GB


[2025-05-08 02:28:49] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-08 02:28:49] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

[2025-05-08 02:28:52] KV Cache is allocated. #tokens: 20480, K size: 0.55 GB, V size: 0.55 GB
[2025-05-08 02:28:52] Memory pool end. avail mem=37.93 GB


[2025-05-08 02:28:53] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-08 02:28:54] INFO:     Started server process [4077047]
[2025-05-08 02:28:54] INFO:     Waiting for application startup.
[2025-05-08 02:28:54] INFO:     Application startup complete.
[2025-05-08 02:28:54] INFO:     Uvicorn running on http://0.0.0.0:32325 (Press CTRL+C to quit)


[2025-05-08 02:28:55] INFO:     127.0.0.1:40672 - "GET /v1/models HTTP/1.1" 200 OK
[2025-05-08 02:28:55] INFO:     127.0.0.1:40684 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-08 02:28:55] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 02:28:57] INFO:     127.0.0.1:40694 - "POST /generate HTTP/1.1" 200 OK
[2025-05-08 02:28:57] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Please generate the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-08 02:29:00] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 02:29:02] Decode batch. #running-req: 1, #token: 52, token usage: 0.00, gen throughput (token/s): 4.46, #queue-req: 0


[2025-05-08 02:29:03] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, gen throughput (token/s): 107.29, #queue-req: 0


[2025-05-08 02:29:03] Decode batch. #running-req: 1, #token: 132, token usage: 0.01, gen throughput (token/s): 106.40, #queue-req: 0


[2025-05-08 02:29:03] Decode batch. #running-req: 1, #token: 172, token usage: 0.01, gen throughput (token/s): 104.76, #queue-req: 0


...

[2025-05-08 02:29:24] Decode batch. #running-req: 1, #token: 1932, token usage: 0.09, gen throughput (token/s): 103.35, #queue-req: 0


[2025-05-08 02:29:24] Decode batch. #running-req: 1, #token: 1972, token usage: 0.10, gen throughput (token/s): 83.35, #queue-req: 0


[2025-05-08 02:29:25] Decode batch. #running-req: 1, #token: 2012, token usage: 0.10, gen throughput (token/s): 102.23, #queue-req: 0


[2025-05-08 02:29:25] Decode batch. #running-req: 1, #token: 2052, token usage: 0.10, gen throughput (token/s): 72.46, #queue-req: 0


[2025-05-08 02:29:26] INFO:     127.0.0.1:56966 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Give me the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-08 02:29:26] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 02:29:26] Decode batch. #running-req: 1, #token: 44, token usage: 0.00, gen throughput (token/s): 52.70, #queue-req: 0


[2025-05-08 02:29:27] Decode batch. #running-req: 1, #token: 84, token usage: 0.00, gen throughput (token/s): 65.35, #queue-req: 0


[2025-05-08 02:29:27] Decode batch. #running-req: 1, #token: 124, token usage: 0.01, gen throughput (token/s): 68.70, #queue-req: 0


[2025-05-08 02:29:28] Decode batch. #running-req: 1, #token: 164, token usage: 0.01, gen throughput (token/s): 92.39, #queue-req: 0


...

[2025-05-08 02:29:45] Decode batch. #running-req: 1, #token: 1924, token usage: 0.09, gen throughput (token/s): 104.80, #queue-req: 0


[2025-05-08 02:29:45] Decode batch. #running-req: 1, #token: 1964, token usage: 0.10, gen throughput (token/s): 103.30, #queue-req: 0


[2025-05-08 02:29:45] Decode batch. #running-req: 1, #token: 2004, token usage: 0.10, gen throughput (token/s): 101.98, #queue-req: 0


[2025-05-08 02:29:46] Decode batch. #running-req: 1, #token: 2044, token usage: 0.10, gen throughput (token/s): 103.09, #queue-req: 0


[2025-05-08 02:29:46] INFO:     127.0.0.1:56966 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "user",
            "content": "Give me the information of the capital of France.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-08 02:29:46] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 02:29:46] Decode batch. #running-req: 1, #token: 39, token usage: 0.00, gen throughput (token/s): 90.56, #queue-req: 0


[2025-05-08 02:29:47] Decode batch. #running-req: 1, #token: 79, token usage: 0.00, gen throughput (token/s): 105.61, #queue-req: 0


[2025-05-08 02:29:47] Decode batch. #running-req: 1, #token: 119, token usage: 0.01, gen throughput (token/s): 105.63, #queue-req: 0
[2025-05-08 02:29:47] INFO:     127.0.0.1:56966 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-08 02:29:47] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 02:29:47] Decode batch. #running-req: 1, #token: 33, token usage: 0.00, gen throughput (token/s): 98.87, #queue-req: 0


[2025-05-08 02:29:48] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, gen throughput (token/s): 107.73, #queue-req: 0


[2025-05-08 02:29:48] Decode batch. #running-req: 1, #token: 113, token usage: 0.01, gen throughput (token/s): 108.85, #queue-req: 0


[2025-05-08 02:29:49] Decode batch. #running-req: 1, #token: 153, token usage: 0.01, gen throughput (token/s): 107.10, #queue-req: 0


[2025-05-08 02:29:49] Decode batch. #running-req: 1, #token: 193, token usage: 0.01, gen throughput (token/s): 93.64, #queue-req: 0


[2025-05-08 02:29:49] Decode batch. #running-req: 1, #token: 233, token usage: 0.01, gen throughput (token/s): 105.51, #queue-req: 0


[2025-05-08 02:29:50] Decode batch. #running-req: 1, #token: 273, token usage: 0.01, gen throughput (token/s): 105.57, #queue-req: 0


[2025-05-08 02:29:50] Decode batch. #running-req: 1, #token: 313, token usage: 0.02, gen throughput (token/s): 106.00, #queue-req: 0


[2025-05-08 02:29:50] Decode batch. #running-req: 1, #token: 353, token usage: 0.02, gen throughput (token/s): 105.59, #queue-req: 0
[2025-05-08 02:29:51] INFO:     127.0.0.1:56966 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "user",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-08 02:29:51] Prefill batch. #new-seq: 1, #new-token: 471, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 02:29:51] Decode batch. #running-req: 1, #token: 495, token usage: 0.02, gen throughput (token/s): 46.29, #queue-req: 0


[2025-05-08 02:29:52] Decode batch. #running-req: 1, #token: 535, token usage: 0.03, gen throughput (token/s): 105.42, #queue-req: 0


[2025-05-08 02:29:52] Decode batch. #running-req: 1, #token: 575, token usage: 0.03, gen throughput (token/s): 105.31, #queue-req: 0


[2025-05-08 02:29:52] Decode batch. #running-req: 1, #token: 615, token usage: 0.03, gen throughput (token/s): 105.98, #queue-req: 0


[2025-05-08 02:29:53] Decode batch. #running-req: 1, #token: 655, token usage: 0.03, gen throughput (token/s): 105.22, #queue-req: 0


[2025-05-08 02:29:53] Decode batch. #running-req: 1, #token: 695, token usage: 0.03, gen throughput (token/s): 103.47, #queue-req: 0


[2025-05-08 02:29:54] Decode batch. #running-req: 1, #token: 735, token usage: 0.04, gen throughput (token/s): 100.54, #queue-req: 0


[2025-05-08 02:29:54] Decode batch. #running-req: 1, #token: 775, token usage: 0.04, gen throughput (token/s): 100.45, #queue-req: 0


[2025-05-08 02:29:54] Decode batch. #running-req: 1, #token: 815, token usage: 0.04, gen throughput (token/s): 98.74, #queue-req: 0
[2025-05-08 02:29:54] INFO:     127.0.0.1:56966 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "user",
        "content": "Here is the information of the capital of France in the JSON format.\n",
    }
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-05-08 02:29:55] Prefill batch. #new-seq: 1, #new-token: 19, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 02:29:55] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, gen throughput (token/s): 40.25, #queue-req: 0


[2025-05-08 02:29:56] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, gen throughput (token/s): 104.90, #queue-req: 0


[2025-05-08 02:29:56] Decode batch. #running-req: 1, #token: 135, token usage: 0.01, gen throughput (token/s): 104.18, #queue-req: 0


[2025-05-08 02:29:57] Decode batch. #running-req: 1, #token: 175, token usage: 0.01, gen throughput (token/s): 103.30, #queue-req: 0


...

[2025-05-08 02:30:14] Decode batch. #running-req: 1, #token: 1895, token usage: 0.09, gen throughput (token/s): 99.75, #queue-req: 0


[2025-05-08 02:30:14] Decode batch. #running-req: 1, #token: 1935, token usage: 0.09, gen throughput (token/s): 102.80, #queue-req: 0


[2025-05-08 02:30:15] Decode batch. #running-req: 1, #token: 1975, token usage: 0.10, gen throughput (token/s): 100.78, #queue-req: 0


[2025-05-08 02:30:15] Decode batch. #running-req: 1, #token: 2015, token usage: 0.10, gen throughput (token/s): 98.20, #queue-req: 0


[2025-05-08 02:30:15] Decode batch. #running-req: 1, #token: 2055, token usage: 0.10, gen throughput (token/s): 101.50, #queue-req: 0
[2025-05-08 02:30:16] INFO:     127.0.0.1:46590 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so I need to provide the information about the capital of France in JSON format. Hmm, I\'m not entirely sure about all the details, but I\'ll try to think it through.\n\nFirst, I know that the capital of France is Paris. That\'s pretty much a given, right? But I should double-check that. Maybe I can recall any other capitals I know. London is the capital of the UK, Rome is Italy, and maybe Tokyo is Japan\'s. Yeah, Paris seems correct for France.\n\nNow, moving on to the population. I think Paris is a very large city, but I\'m not sure of the exact number. I remember it\'s over 3 million, but I\'m not certain. Maybe around 3.5 million? I should probably look that up, but since I can\'t right now, I\'ll go with 3,500,000 as an estimate.\n\nNext, the area. Par

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(text, tokenize=False, add_generation_prompt=True)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-05-08 02:30:16] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 02:30:16] Decode batch. #running-req: 1, #token: 32, token usage: 0.00, gen throughput (token/s): 100.14, #queue-req: 0


[2025-05-08 02:30:16] Decode batch. #running-req: 1, #token: 72, token usage: 0.00, gen throughput (token/s): 103.55, #queue-req: 0


[2025-05-08 02:30:17] Decode batch. #running-req: 1, #token: 112, token usage: 0.01, gen throughput (token/s): 106.04, #queue-req: 0


[2025-05-08 02:30:17] Decode batch. #running-req: 1, #token: 152, token usage: 0.01, gen throughput (token/s): 107.48, #queue-req: 0


...

[2025-05-08 02:30:21] Decode batch. #running-req: 1, #token: 632, token usage: 0.03, gen throughput (token/s): 105.30, #queue-req: 0


[2025-05-08 02:30:22] Decode batch. #running-req: 1, #token: 672, token usage: 0.03, gen throughput (token/s): 90.57, #queue-req: 0


[2025-05-08 02:30:22] Decode batch. #running-req: 1, #token: 712, token usage: 0.03, gen throughput (token/s): 105.71, #queue-req: 0


[2025-05-08 02:30:23] Decode batch. #running-req: 1, #token: 752, token usage: 0.04, gen throughput (token/s): 107.56, #queue-req: 0


[2025-05-08 02:30:23] Decode batch. #running-req: 1, #token: 792, token usage: 0.04, gen throughput (token/s): 104.81, #queue-req: 0
[2025-05-08 02:30:23] INFO:     127.0.0.1:49726 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-05-08 02:30:23] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-08 02:30:23] Prefill batch. #new-seq: 3, #new-token: 3, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 02:30:24] Decode batch. #running-req: 3, #token: 119, token usage: 0.01, gen throughput (token/s): 174.56, #queue-req: 0


[2025-05-08 02:30:24] Decode batch. #running-req: 3, #token: 239, token usage: 0.01, gen throughput (token/s): 305.83, #queue-req: 0


[2025-05-08 02:30:24] Decode batch. #running-req: 3, #token: 359, token usage: 0.02, gen throughput (token/s): 306.83, #queue-req: 0


[2025-05-08 02:30:25] Decode batch. #running-req: 3, #token: 479, token usage: 0.02, gen throughput (token/s): 306.86, #queue-req: 0


[2025-05-08 02:30:25] Decode batch. #running-req: 3, #token: 599, token usage: 0.03, gen throughput (token/s): 304.92, #queue-req: 0
[2025-05-08 02:30:25] INFO:     127.0.0.1:49736 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThat's all the information I have.\n\nOkay, so I need to figure out the capital of France. I know that Paris is the capital, but I'm not entirely sure. Let me think about why I think that. I've heard it mentioned a lot, especially in movies and TV shows. People often go there for business or tourism. Also, I remember learning in school that Paris is a major city in France, known for landmarks like the Eiffel Tower and the Louvre Museum. Those places are famous worldwide, which makes me think that Paris is indeed the capital. Maybe I can cross-check this with some other sources or my notes. Wait, I don't have any other information right now, but based on what I know, Paris is the capital of France. I don't recall any other majo

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-05-08 02:30:25] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 02:30:26] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 127.56, #queue-req: 0


[2025-05-08 02:30:26] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 107.96, #queue-req: 0


[2025-05-08 02:30:26] Decode batch. #running-req: 1, #token: 121, token usage: 0.01, gen throughput (token/s): 106.69, #queue-req: 0


[2025-05-08 02:30:27] Decode batch. #running-req: 1, #token: 161, token usage: 0.01, gen throughput (token/s): 107.22, #queue-req: 0


...

[2025-05-08 02:30:43] Decode batch. #running-req: 1, #token: 1881, token usage: 0.09, gen throughput (token/s): 104.32, #queue-req: 0


[2025-05-08 02:30:43] Decode batch. #running-req: 1, #token: 1921, token usage: 0.09, gen throughput (token/s): 100.34, #queue-req: 0


[2025-05-08 02:30:44] Decode batch. #running-req: 1, #token: 1961, token usage: 0.10, gen throughput (token/s): 98.53, #queue-req: 0


[2025-05-08 02:30:44] Decode batch. #running-req: 1, #token: 2001, token usage: 0.10, gen throughput (token/s): 99.66, #queue-req: 0


[2025-05-08 02:30:45] Decode batch. #running-req: 1, #token: 2041, token usage: 0.10, gen throughput (token/s): 103.72, #queue-req: 0
[2025-05-08 02:30:45] INFO:     127.0.0.1:50924 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\(

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-05-08 02:30:45] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 19, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 02:30:45] Decode batch. #running-req: 1, #token: 47, token usage: 0.00, gen throughput (token/s): 98.38, #queue-req: 0


[2025-05-08 02:30:45] Decode batch. #running-req: 1, #token: 87, token usage: 0.00, gen throughput (token/s): 105.69, #queue-req: 0


[2025-05-08 02:30:46] Decode batch. #running-req: 1, #token: 127, token usage: 0.01, gen throughput (token/s): 105.74, #queue-req: 0


[2025-05-08 02:30:46] Decode batch. #running-req: 1, #token: 167, token usage: 0.01, gen throughput (token/s): 105.28, #queue-req: 0


...

[2025-05-08 02:30:52] Decode batch. #running-req: 1, #token: 727, token usage: 0.04, gen throughput (token/s): 106.56, #queue-req: 0


[2025-05-08 02:30:52] Decode batch. #running-req: 1, #token: 767, token usage: 0.04, gen throughput (token/s): 104.63, #queue-req: 0


[2025-05-08 02:30:52] Decode batch. #running-req: 1, #token: 807, token usage: 0.04, gen throughput (token/s): 104.74, #queue-req: 0


[2025-05-08 02:30:53] Decode batch. #running-req: 1, #token: 847, token usage: 0.04, gen throughput (token/s): 105.29, #queue-req: 0


[2025-05-08 02:30:53] Decode batch. #running-req: 1, #token: 887, token usage: 0.04, gen throughput (token/s): 103.89, #queue-req: 0
[2025-05-08 02:30:53] INFO:     127.0.0.1:52100 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

[2025-05-08 02:30:53] Child process unexpectedly failed with an exit code 9. pid=4077404
[2025-05-08 02:30:53] Child process unexpectedly failed with an exit code 9. pid=4077337


## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.34s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.24s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.26s/it]



### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure, here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Tiananmen Square",
    "Forbidden City",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
The capital of France is Paris. It is located in the northern part of the country, on the edge of the Seine River. Paris is a major city in France, known for its rich history, landmarks, and serves as the headquarters of the French government, the National Assembly, and the Supreme Court.

Okay, so the user just asked me to provide information about the capital of France. I should make sure to give a comprehensive yet concise response. They might be a student starting their research or someone just curious. It's important to highlight Paris's significance, including its geographical location, historical importance, and the various institutions based there
Prompt: Give me the information of the capital of Germany.
Generated text:  a. What is the name of the capital? b. In which continent is it located? c. What is the official language? d. What is the currency used in the capital?

a. The capital of Germany is Ber

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its history, culture, economy, and environment.

**Question:** What is London's nickname, and why is it called that?

**Question:** What is London's population figure?

**Question:** In terms of language, what is London's dominant language?

**Question:** What are the primary industries in London's economy?

**Question:** How does London's economy function overall?

**Question:** What is the capital city of England?

**Question:** What is the significance of the Tower of London?

**Question:** How has London evolved over time in terms of its urban structure?

**Question:** What is the weather like in London?

**Question
Prompt: Please provide information about Paris as a major global city:
Generated text:  its history, cultural significance, and current status.


Paris is one of the most important cities in the world, serving as the capital of France. It has a rich history dating back to prehistory

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜User｜>Here is the information of the capital of France in the JSON format.
<｜Assistant｜><think>

Generated text: Alright, the user is asking for the information about the capital of France in JSON format. Hmm, I should make sure I provide the correct and detailed information. First, I need to recall that the capital of France is Paris. Okay, that's straightforward. Now, I should think about what other details are relevant for a JSON response. 

Maybe include the population, which I believe is around 2 million. The area should be approximately 105 square kilometers. As for the official languages, I remember it's French and a few others like English, Spanish, and Arabic. The official currency is the euro. 

I should structure this information neatly in a JSON format. I'll create an object with a "country" key, and then another object inside it for "capital_info". This inner object can have "city" as the name, "population" as the figure, "area" in square kil

In [19]:
llm.shutdown()